In [1]:
import pandas as pd

In [2]:
meta_muts = pd.read_pickle('./targ_mut_df.pkl')
display(len(meta_muts), meta_muts.head())

72

,exp,sample,Position,Mutation Type,Sequence Change,Details,Gene,ALEdb source,genetic,clean genes,bnums,genes of interest
424755,BOP_27_pfkA,2 33 1 1,2661691,SNP,C→A,A147A (GCG→GCT),iscR,private,True,{iscR},{b2531},{b2531}
455336,Bop_27_TPP,1 36 1 1,2661664,SNP,G→A,I156I (ATC→ATT),iscR,private,True,{iscR},{b2531},{b2531}
468514,GLU_16321-M9-NC_000913_3gb-stationary-37-Gluco...,11 20 0 1,2661969,SNP,C→G,V55L (GTT→CTT),iscR,private,True,{iscR},{b2531},{b2531}
468596,GLU_16321-M9-NC_000913_3gb-stationary-37-Gluco...,13 25 0 1,2661969,SNP,C→G,V55L (GTT→CTT),iscR,private,True,{iscR},{b2531},{b2531}
468716,GLU_16321-M9-NC_000913_3gb-stationary-37-Gluco...,17 23 0 1,2661969,SNP,C→G,V55L (GTT→CTT),iscR,private,True,{iscR},{b2531},{b2531}


In [3]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from aledbmutil.mut import get_mut_size, get_mob_size, is_genetic_mut, is_coding_mut, predict_mutation_effect_on_feature, get_SNP_aa_pos, get_DEL_INS_MOB_aa_start_pos, get_DEL_AA_range


meta_muts["genetic"] = meta_muts["Details"].apply(is_genetic_mut)
meta_muts["coding"] = meta_muts["Details"].apply(is_coding_mut)
meta_muts["mutation size"] = meta_muts.apply(get_mut_size, axis=1)  # get_mut_size currently doesn't return MOB mut sizes. Isn't something currently necessary.

In [4]:
from aledbmutil.mut import get_SNP_aa_pos, get_DEL_AA_range

def _get_SNP_AA_range(mut_details_str):
    SNP_AA_range = ()
    AA_pos = get_SNP_aa_pos(mut_details_str)
    SNP_AA_range = (AA_pos, AA_pos)
    return SNP_AA_range

meta_muts["AA range"] = None
# meta_muts["AA range"] = meta_muts.apply(
#     lambda r: _get_SNP_AA_range(r["Details"]) if ((r["Mutation Type"] == "SNP") & (r["coding"]==1)) else r["AA range"],
#     axis=1)
meta_muts["AA range"] = meta_muts.apply(lambda r:
                                        _get_SNP_AA_range(r["Details"]) if ((r["Mutation Type"] == "SNP") & (r["coding"]==1)) \
                                        else (get_DEL_AA_range(r["Details"]) if ((r["Mutation Type"] == "DEL") & (r["coding"]==1)) \
                                        else r["AA range"]),
                                        axis=1)
meta_muts.head()

,exp,sample,Position,Mutation Type,Sequence Change,Details,Gene,ALEdb source,genetic,clean genes,bnums,genes of interest,coding,mutation size,AA range
424755,BOP_27_pfkA,2 33 1 1,2661691,SNP,C→A,A147A (GCG→GCT),iscR,private,True,{iscR},{b2531},{b2531},True,1,"(147, 147)"
455336,Bop_27_TPP,1 36 1 1,2661664,SNP,G→A,I156I (ATC→ATT),iscR,private,True,{iscR},{b2531},{b2531},True,1,"(156, 156)"
468514,GLU_16321-M9-NC_000913_3gb-stationary-37-Gluco...,11 20 0 1,2661969,SNP,C→G,V55L (GTT→CTT),iscR,private,True,{iscR},{b2531},{b2531},True,1,"(55, 55)"
468596,GLU_16321-M9-NC_000913_3gb-stationary-37-Gluco...,13 25 0 1,2661969,SNP,C→G,V55L (GTT→CTT),iscR,private,True,{iscR},{b2531},{b2531},True,1,"(55, 55)"
468716,GLU_16321-M9-NC_000913_3gb-stationary-37-Gluco...,17 23 0 1,2661969,SNP,C→G,V55L (GTT→CTT),iscR,private,True,{iscR},{b2531},{b2531},True,1,"(55, 55)"


In [5]:
# currently only tracking seq changes with SNPs for ALEdb mutations for later processing with AA property change trends
meta_muts['AA ref seq'] = meta_muts.apply(lambda m: m['Details'][0] if m["Mutation Type"]=='SNP' else '', axis=1)
meta_muts['AA seq change'] = meta_muts.apply(lambda m: m['Details'].split(' (')[0][-1] if m["Mutation Type"]=='SNP' else '', axis=1)
meta_muts.head()

,exp,sample,Position,Mutation Type,Sequence Change,Details,Gene,ALEdb source,genetic,clean genes,bnums,genes of interest,coding,mutation size,AA range,AA ref seq,AA seq change
424755,BOP_27_pfkA,2 33 1 1,2661691,SNP,C→A,A147A (GCG→GCT),iscR,private,True,{iscR},{b2531},{b2531},True,1,"(147, 147)",A,A
455336,Bop_27_TPP,1 36 1 1,2661664,SNP,G→A,I156I (ATC→ATT),iscR,private,True,{iscR},{b2531},{b2531},True,1,"(156, 156)",I,I
468514,GLU_16321-M9-NC_000913_3gb-stationary-37-Gluco...,11 20 0 1,2661969,SNP,C→G,V55L (GTT→CTT),iscR,private,True,{iscR},{b2531},{b2531},True,1,"(55, 55)",V,L
468596,GLU_16321-M9-NC_000913_3gb-stationary-37-Gluco...,13 25 0 1,2661969,SNP,C→G,V55L (GTT→CTT),iscR,private,True,{iscR},{b2531},{b2531},True,1,"(55, 55)",V,L
468716,GLU_16321-M9-NC_000913_3gb-stationary-37-Gluco...,17 23 0 1,2661969,SNP,C→G,V55L (GTT→CTT),iscR,private,True,{iscR},{b2531},{b2531},True,1,"(55, 55)",V,L


In [6]:
exp_sel_pres_d = {
    'ROS_eAMP': "paraquat",  # starting strains were also pre-evolved on glucose
 }

meta_muts['selection pressure'] = meta_muts.apply(lambda r: 'unpublished' if r['ALEdb source'] == 'private' else '', axis=1)
meta_muts['selection pressure'] = meta_muts.exp.apply(lambda e: exp_sel_pres_d[e] if e in exp_sel_pres_d.keys() else 'unpublished')
assert('' not in set(meta_muts['selection pressure']))

In [7]:
# TODO: fix warnings with append
import warnings
warnings.filterwarnings('ignore')


aa_var_df = pd.DataFrame()

from aledbmutil.mut import get_mut_size  #, get_DEL_INS_MOB_aa_start_pos


for mt, mut_type_mut_df in meta_muts.groupby("Mutation Type"):
    assert mt != "MOB", 'Currently not handilng MOB. The below INS logic might be able to handle this.'
    if mt == "SNP":
        for _, m in mut_type_mut_df.iterrows():
            mut_eff = ""
            details_str = m["Details"]
            details_str = details_str[0:details_str.find(" (")]
            
            if details_str[-1] == '*':  # Could have them show the effected downstream positions like with ACS SynBio paper. 
                mut_eff = "truncation"
            elif details_str[0] == details_str[-1]:
                mut_eff = "synonymous substitution"
            elif details_str[0] != details_str[-1]:
                mut_eff = "nonsynonymous substitution"
            else:
                mut_eff = "unknown"
            
            aa_pos = int(details_str[1:-1])
            aa_var_df= aa_var_df.append({
                "AA pos": aa_pos,
                "mutation type": "SUB",
                "mutation effect": mut_eff,
                "mutation source": m['selection pressure'],
                'AA ref seq': m['AA ref seq'],
                'AA seq change': m['AA seq change'],
            }, ignore_index=True)
    if (mt == 'DEL'):  # Unravel (unless truncation) since downstream analysis just looks at individual mutated AAs no matter if they come from the same allele.
        display(mut_type_mut_df)
        for _, m in mut_type_mut_df.iterrows():
            mut_eff = "deletion"
            mut_size = m['mutation size']
            if mut_size % 3 != 0:  # Don't want to record all positions involved in the truncation, just the first.
                aa_var_df= aa_var_df.append({
                    "AA pos": m['AA range'][0],
                    "mutation type": mt,
                    "mutation effect": "truncation",
                    "mutation source": m['selection pressure'],
                    'AA ref seq': '',
                    'AA seq change': '',
                },ignore_index=True)
            else:                
                for aa_pos in range(m['AA range'][0], m['AA range'][1]+1):
                    aa_var_df= aa_var_df.append({
                        "AA pos": aa_pos,
                        "mutation type": mt,
                        "mutation effect": mut_eff,
                        "mutation source": m['selection pressure'],
                        'AA ref seq': '',
                        'AA seq change': '',
                    },ignore_index=True)
    if (mt == 'INS'):
        for _, m in mut_type_mut_df.iterrows():
            mut_eff = "insertion"  # currently assuming only insertions remain
            mut_size = m['mutation size']
            if mut_size % 3 != 0:
                mut_eff = "truncation"
            aa_var_df= aa_var_df.append({
                "AA pos": get_DEL_INS_MOB_aa_start_pos(m["Details"]),
                "mutation type": mt,
                "mutation effect": mut_eff,
                "mutation source": m['selection pressure'],
                'AA ref seq': '',
                'AA seq change': '',
            },ignore_index=True)
    if (mt == 'SUB'):  # Unravel since downstream analysis just looks at individual mutated AAs no matter if they come from the same allele.
        for _, m in mut_type_mut_df.iterrows():
            for aa_pos in range(m['AA range'][0], m['AA range'][1]+1):
                aa_var_df= aa_var_df.append({
                    "AA pos": aa_pos,
                    "mutation type": mt,
                    "mutation effect": "nonsynonymous substitution",
                    "mutation source": m['selection pressure'],
                    'AA ref seq': '',
                    'AA seq change': '',
                },ignore_index=True)


display(len(aa_var_df), aa_var_df.head())

,exp,sample,Position,Mutation Type,Sequence Change,Details,Gene,ALEdb source,genetic,clean genes,bnums,genes of interest,coding,mutation size,AA range,AA ref seq,AA seq change,selection pressure
445232,MUSE-M9-NC_000913_3gb-stationary-37-Glucose2-N...,33 29 1 1,2661730,DEL,Δ5 bp,coding (398‐402/489 nt),iscR,private,True,{iscR},{b2531},{b2531},True,5,"(133, 134)",,,unpublished
445493,MUSE-M9-NC_000913_3gb-stationary-37-Glucose2-N...,39 66 1 1,2661844,DEL,Δ9 bp,coding (280‐288/489 nt),iscR,private,True,{iscR},{b2531},{b2531},True,9,"(94, 96)",,,unpublished
424924,rpoB_E672K-M9-NC_000913_3gb-stationary-37-Gluc...,10 12 1 1,2661831,DEL,Δ9 bp,coding (293‐301/489 nt),iscR,private,True,{iscR},{b2531},{b2531},True,9,"(98, 101)",,,unpublished
424977,rpoB_E672K-M9-NC_000913_3gb-stationary-37-Gluc...,12 4 0 1,2661831,DEL,Δ9 bp,coding (293‐301/489 nt),iscR,private,True,{iscR},{b2531},{b2531},True,9,"(98, 101)",,,unpublished


80

,AA pos,mutation type,mutation effect,mutation source,AA ref seq,AA seq change
0,133,DEL,truncation,unpublished,,
1,94,DEL,deletion,unpublished,,
2,95,DEL,deletion,unpublished,,
3,96,DEL,deletion,unpublished,,
4,98,DEL,deletion,unpublished,,


In [8]:
aa_var_cnt_df = pd.DataFrame()
gcols = ['AA pos', 'mutation type', 'mutation effect', 'mutation source', 'AA ref seq', 'AA seq change']
for g, gdf in aa_var_df.groupby(gcols):  # Essentially just finding duplicates
    d = dict(zip(gcols, g))
    d['mutation count'] = len(gdf)
    aa_var_cnt_df = aa_var_cnt_df.append(d, ignore_index=True)
aa_var_cnt_df.head()

,AA pos,mutation type,mutation effect,mutation source,AA ref seq,AA seq change,mutation count
0,13,SUB,nonsynonymous substitution,paraquat,A,T,1
1,13,SUB,nonsynonymous substitution,unpublished,A,T,1
2,25,SUB,nonsynonymous substitution,unpublished,P,Q,1
3,33,SUB,nonsynonymous substitution,unpublished,E,D,1
4,37,SUB,nonsynonymous substitution,paraquat,I,V,1


In [9]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import const


aa_var_cnt_df['mutation effect color'] = aa_var_cnt_df['mutation effect'].apply(lambda e: const.MUTATION_TYPE_COLOR_PALETTE[e])
aa_var_cnt_df.head()

,AA pos,mutation type,mutation effect,mutation source,AA ref seq,AA seq change,mutation count,mutation effect color
0,13,SUB,nonsynonymous substitution,paraquat,A,T,1,#DD8452
1,13,SUB,nonsynonymous substitution,unpublished,A,T,1,#DD8452
2,25,SUB,nonsynonymous substitution,unpublished,P,Q,1,#DD8452
3,33,SUB,nonsynonymous substitution,unpublished,E,D,1,#DD8452
4,37,SUB,nonsynonymous substitution,paraquat,I,V,1,#DD8452


In [10]:
sel_pres_clr_d = {
    # 'FeSO4': '#729ECE',
    # 'Homo sapiens tpi': '#FF9E4A',
    # 'Pyrobaculum aerophilum pgi': '#67BF5C',
    # 'Pyrobaculum aerophilum tpi': '#ED665D',
    # 'Vibrio cholerae tpi': '#AD8BC9',
    # 'isobutanol': '#ED97CA',
    'paraquat': '#6DCCDA',
    'unpublished': '#AD8BC9'
 }

aa_var_cnt_df['selection pressure color'] = aa_var_cnt_df['mutation source'].apply(lambda e: sel_pres_clr_d[e])
aa_var_cnt_df

,AA pos,mutation type,mutation effect,mutation source,AA ref seq,AA seq change,mutation count,mutation effect color,selection pressure color
0,13,SUB,nonsynonymous substitution,paraquat,A,T,1,#DD8452,#6DCCDA
1,13,SUB,nonsynonymous substitution,unpublished,A,T,1,#DD8452,#AD8BC9
2,25,SUB,nonsynonymous substitution,unpublished,P,Q,1,#DD8452,#AD8BC9
3,33,SUB,nonsynonymous substitution,unpublished,E,D,1,#DD8452,#AD8BC9
4,37,SUB,nonsynonymous substitution,paraquat,I,V,1,#DD8452,#6DCCDA
5,37,SUB,nonsynonymous substitution,unpublished,I,V,1,#DD8452,#AD8BC9
6,40,INS,truncation,unpublished,,,1,#C44E52,#AD8BC9
7,43,SUB,nonsynonymous substitution,unpublished,E,G,10,#DD8452,#AD8BC9
8,43,SUB,nonsynonymous substitution,unpublished,E,V,1,#DD8452,#AD8BC9
9,44,SUB,nonsynonymous substitution,unpublished,Q,K,1,#DD8452,#AD8BC9


In [11]:
def _get_variant_name(variant):
    var_name = ''
    if variant['mutation type'] == 'SUB':
        var_name = variant['mutation type'] + " " + variant['AA ref seq'] + str(variant['AA pos']) + variant['AA seq change']
    if (variant['mutation type'] == 'DEL') or (variant['mutation type'] == 'INS'):
        var_name = variant['mutation type'] + " " + str(variant['AA pos'])
    return var_name


aa_var_cnt_df['name'] = aa_var_cnt_df.apply(lambda r: _get_variant_name(r), axis=1)
aa_var_cnt_df

,AA pos,mutation type,mutation effect,mutation source,AA ref seq,AA seq change,mutation count,mutation effect color,selection pressure color,name
0,13,SUB,nonsynonymous substitution,paraquat,A,T,1,#DD8452,#6DCCDA,SUB A13T
1,13,SUB,nonsynonymous substitution,unpublished,A,T,1,#DD8452,#AD8BC9,SUB A13T
2,25,SUB,nonsynonymous substitution,unpublished,P,Q,1,#DD8452,#AD8BC9,SUB P25Q
3,33,SUB,nonsynonymous substitution,unpublished,E,D,1,#DD8452,#AD8BC9,SUB E33D
4,37,SUB,nonsynonymous substitution,paraquat,I,V,1,#DD8452,#6DCCDA,SUB I37V
5,37,SUB,nonsynonymous substitution,unpublished,I,V,1,#DD8452,#AD8BC9,SUB I37V
6,40,INS,truncation,unpublished,,,1,#C44E52,#AD8BC9,INS 40
7,43,SUB,nonsynonymous substitution,unpublished,E,G,10,#DD8452,#AD8BC9,SUB E43G
8,43,SUB,nonsynonymous substitution,unpublished,E,V,1,#DD8452,#AD8BC9,SUB E43V
9,44,SUB,nonsynonymous substitution,unpublished,Q,K,1,#DD8452,#AD8BC9,SUB Q44K


In [12]:
aa_var_cnt_df.to_csv('iscR_ale_muts.csv', index=False)

In [13]:
mut_needle_plot_legend_df = pd.DataFrame()
mut_needle_plot_legend_df = aa_var_cnt_df.copy()
mut_needle_plot_legend_df = mut_needle_plot_legend_df[['mutation source', 'selection pressure color', ]]
mut_needle_plot_legend_df.rename(columns={'mutation source': 'name', 'selection pressure color': 'fill'}, inplace=True)
mut_needle_plot_legend_df['edge'] = 'white'
mut_needle_plot_legend_df.drop_duplicates(inplace=True)

df = aa_var_cnt_df.copy()
df = df[['mutation effect', 'mutation effect color']]
df.rename(columns={'mutation effect': 'name', 'mutation effect color': 'edge'}, inplace=True)
df.drop_duplicates(inplace=True)
df['fill'] = 'white'
mut_needle_plot_legend_df = mut_needle_plot_legend_df.append(df, ignore_index=True)
display(mut_needle_plot_legend_df)

mut_needle_plot_legend_df.to_csv('iscR_ale_muts_legend.csv', index=False)

,name,fill,edge
0,paraquat,#6DCCDA,white
1,unpublished,#AD8BC9,white
2,nonsynonymous substitution,white,#DD8452
3,truncation,white,#C44E52
4,deletion,white,#937860
5,synonymous substitution,white,#55A868


In [14]:
feats_df = pd.read_pickle('./annot_df.pkl')

# filtering out features that aren't useful
feats_to_remove = [
    'Initiator methionine',
    'HTH-type transcriptional regulator IscR [iscR]',
    'Transcription regulator Rrf2',
    'HTH-TYPE TRANSCRIPTIONAL REGULATOR ISCR',
    'WINGED HELIX-TURN-HELIX TRANSCRIPTIONAL REGULATOR, RRF2 FAMILY',
    'Winged helix-like DNA-binding domain superfamily/Winged helix DNA-binding domain',
    'Winged helix DNA-binding domain superfamily',
    'Winged helix DNA-binding domain',
    'Winged helix-like DNA-binding domain superfamily',
    'Iron-dependent Transcriptional regulator',
    'Domain',
    # 'Rrf2-type HTH domain profile',
    'Transcription regulator Rrf2-type, conserved site',
    'Rrf2 family protein',
    'Transcription factor HTH,  IscR',
    # 'Chain',
    'Disordered region',
    'iron-sulfur cluster assembly transcription factor IscR',  # redundant once 'Disordered region' is removed
]
feats_df = feats_df[~(feats_df['feature'].isin(feats_to_remove))]
feats_df.reset_index(inplace=True, drop=True)
feats_df

feats_df.head()

,feature,start,end,range
0,Chain,1,162,"(1, 162)"
1,IscR subunit interface,132,135,"(132, 4)"
2,Rrf2-type HTH domain profile,2,131,"(2, 130)"
3,IscR subunit interface,128,130,"(128, 3)"
4,IscR subunit interface,124,124,"(124, 1)"


In [15]:
layer = 0
layer_d = dict()
for f in feats_df.feature.unique():
    layer_d[f] = layer
    layer += 1
feats_df['layer'] = feats_df['feature'].apply(lambda f: layer_d[f])
feats_df.head()

,feature,start,end,range,layer
0,Chain,1,162,"(1, 162)",0
1,IscR subunit interface,132,135,"(132, 4)",1
2,Rrf2-type HTH domain profile,2,131,"(2, 130)",2
3,IscR subunit interface,128,130,"(128, 3)",1
4,IscR subunit interface,124,124,"(124, 1)",1


In [16]:
feat_color_d = dict()
muted=["#4878D0", "#EE854A", "#6ACC64", "#D65F5F", "#956CB4", "#8C613C", "#DC7EC0", "#797979", "#D5BB67", "#82C6E2"]  # from seaborn
feats = list(feats_df.feature.unique())
for i in range(0, len(feats_df.feature.unique())):
    feat_color_d[feats[i]] = muted[i]
feat_color_d

feats_df['color'] = feats_df['feature'].apply(lambda f: feat_color_d[f])

feats_df.drop(columns=["range"], inplace=True)
feats_df.drop_duplicates(inplace=True)

display(feats_df, feat_color_d)
feats_df.to_csv('./iscR_feats.csv')

,feature,start,end,layer,color
0,Chain,1,162,0,#4878D0
1,IscR subunit interface,132,135,1,#EE854A
2,Rrf2-type HTH domain profile,2,131,2,#6ACC64
3,IscR subunit interface,128,130,1,#EE854A
4,IscR subunit interface,124,124,1,#EE854A
5,IscR subunit interface,120,121,1,#EE854A
6,IscR subunit interface,116,117,1,#EE854A
7,IscR subunit interface,105,114,1,#EE854A
8,2Fe-2S binding site,104,104,3,#D65F5F
9,IscR subunit interface,102,103,1,#EE854A


{'Chain': '#4878D0',
 'IscR subunit interface': '#EE854A',
 'Rrf2-type HTH domain profile': '#6ACC64',
 '2Fe-2S binding site': '#D65F5F',
 'Rrf2-type HTH domain signature': '#956CB4',
 'HTH DNA binding domain': '#8C613C',
 'SO4 binding site': '#DC7EC0'}

In [17]:
# Filtering out some features that are not useful for the NGL visual 3D analysis
ngl_feat_df = feats_df.copy()
ngl_feat_df = ngl_feat_df[ngl_feat_df['feature'].isin(['2Fe-2S binding site', 'HTH DNA binding domain', 'IscR subunit interface'])]

In [18]:
ngl_feat_df["size"] = ngl_feat_df.apply(lambda r: r["end"] - r["start"] + 1, axis=1)
ngl_feat_df = ngl_feat_df[["feature", "start", "end", "color", "size"]]
ngl_feat_df = ngl_feat_df.sort_values("size", ascending=False)
ngl_feat_df = ngl_feat_df.reset_index(drop=True)  # json explicitly uses the indexes

with open('feats.json', 'w') as outfile:   # for NGL
    outfile.write(ngl_feat_df.to_json())
ngl_feat_df

,feature,start,end,color,size
0,HTH DNA binding domain,28,51,#8C613C,24
1,IscR subunit interface,105,114,#EE854A,10
2,IscR subunit interface,1,5,#EE854A,5
3,IscR subunit interface,33,36,#EE854A,4
4,IscR subunit interface,132,135,#EE854A,4
5,IscR subunit interface,128,130,#EE854A,3
6,IscR subunit interface,116,117,#EE854A,2
7,IscR subunit interface,102,103,#EE854A,2
8,IscR subunit interface,120,121,#EE854A,2
9,IscR subunit interface,19,20,#EE854A,2
